# famCHESS EDA

Kendra Wyant

## Notes

This script reads in the cleaned famCHESS data and performs EDA on final variables.

## Setup

In [ ]:
options(conflicts.policy = "depends.ok")
suppressMessages(library(tidyverse))
suppressMessages(library(janitor))
theme_set(theme_classic()) 

devtools::source_url("https://github.com/jjcurtin/lab_support/blob/main/format_path.R?raw=true")


ℹ SHA-1 hash of file is "a58e57da996d1b70bb9a5b58241325d6fd78890f"

## Data

### Cleaning

In [ ]:
data <- haven::read_sav(here::here(path_data, "Famchess_clean_combined_FINAL.sav")) |> 
  clean_names() 

dsm <- haven::read_sav(here::here(path_data, "FamCHESS_DSM5.sav")) |> 
  clean_names() # for stratification check


Remove participants with inaccurate data (from Olivia’s script):

-   167PT (baseline and 4 month): FAM passed away after baseline
-   208FAM & 208PT (noted on all timepoints): Might be wholly unreliable since we suspect he is his own partner
-   365PT (noted on 1 timepoint): Answered relationship questions about her dog, not her son
-   365FAM (noted on 1 timepoint): Answered relationship questions about romantic partner, not his mom

In [ ]:
data <- data |> 
  filter(!study_id %in% c("167PT", "208FAM", "208PT", "365PT", "365FAM"))


Recode variables (from Olivia’s script)

195PT: reported prefer not to say for gender on baseline but reported male in TLFB data. Changing to male to match TLFB.

In [ ]:
data <- data |> 
  mutate(gender = if_else(study_id == "195PT", 2, gender))


Gender - 2 levels (male / non-male) - TLFB is also coded this way

-   500PT: non-binary
-   128FAM: Transgender women

*KW: currently setting these two people to Non-male, however, I saw this in scales/measures doc - not enough non-man/ women responses to have 3rd level. For patients, those who don’t fall into groups set to missing*

In [ ]:
data <- data |> 
  mutate(gender_original = gender, 
         gender = if_else(gender != 2, 1, gender),
         gender = sjlabelled::set_labels(gender, labels = c("Non-male", "Male"))) 

tabyl(data$gender_original)


 data$gender_original   n     percent valid_percent
                    1 775 0.567765568   0.569434240
                    2 580 0.424908425   0.426157237
                    3   2 0.001465201   0.001469508
                    5   4 0.002930403   0.002939015
                   NA   4 0.002930403            NA

 data$gender   n     percent valid_percent
           1 781 0.572161172     0.5738428
           2 580 0.424908425     0.4261572
          NA   4 0.002930403            NA

Create binary covariate for race/ethnicity defined as: White only (non-Hispanic) vs. not White only

In [ ]:
data <- data |> 
  mutate(race_white_only = if_else(ethnicity == 0 & race_1 == 1 &
                                     race_2 == 0 & race_3 == 0 &
                                     race_4 == 0 & race_5 == 0 &
                                     race_6 == 0, 1, 0))

tabyl(data$race_white_only)


 data$race_white_only    n     percent valid_percent
                    0  335 0.245421245     0.2461425
                    1 1026 0.751648352     0.7538575
                   NA    4 0.002930403            NA

Center non-binary covariates: age and edu

In [ ]:
data <- data |> 
  mutate(age_c = age - mean(age, na.rm = TRUE))


*KW: education is not continuous. Is centering best transformation? Maybe instead pick a certain level?*

In [ ]:
sjlabelled::get_labels(data$edu)


[1] "8th grade or less"                     
[2] "Some high school, but did not graduate"
[3] "High school graduate or GED"           
[4] "Some college or 2 year degree"         
[5] "4-year college graduate"               
[6] "More than 4-year college degree"       

[1] 4.274798

 data$edu   n     percent valid_percent
        1   8 0.005860806   0.005878031
        2  42 0.030769231   0.030859662
        3 266 0.194871795   0.195444526
        4 510 0.373626374   0.374724467
        5 322 0.235897436   0.236590742
        6 213 0.156043956   0.156502572
       NA   4 0.002930403            NA

Recode data to have baseline scores of outcomes as covariate

*KW: 36 study ids provided baseline info but no data at later time points*

In [ ]:
baseline_values <- data |>  
  filter(time == 0) |> 
  select(study_id, 
         hdd_0 = per_heavy_drink_days, 
         pda_0 = per_days_abstinent, 
         oq45_0 = oq45_scored, 
         relationsatisf_0 = relationsatisf_scored, 
         abuse_0 = abuse_scored)

baseline_values |> 
  filter(!study_id %in% subset(data, time != 0)$study_id) |> 
  print(n = Inf)


# A tibble: 36 × 6
   study_id hdd_0 pda_0 oq45_0 relationsatisf_0 abuse_0
   <chr>    <dbl> <dbl>  <dbl>            <dbl>   <dbl>
 1 110PT    100     0       25               25      NA
 2 111PT     30.6  44.6     50               26      NA
 3 117FAM    NA    NA       32               18      NA
 4 119FAM    NA    NA       29               27       3
 5 119PT     98.3   0.8     64               25      NA
 6 121PT     33.9  66.1     68               31      26
 7 125PT     26.4  73.6     74               21      NA
 8 130FAM    NA    NA       94               23       7
 9 130PT     99.2   0.8    109               25       2
10 139PT      9.2  90.8     80               21      NA
11 140FAM    NA    NA       61               20      NA
12 148FAM    NA    NA       70               17       4
13 148PT     99.2   0.8     74               18       2
14 149PT     75    23.3     30               25      NA
15 155FAM    NA    NA       35               17       2
16 155PT     NA    NA       7

In [ ]:
data <- data |> 
  filter(time != 0) |> 
  full_join(baseline_values, by = "study_id")


*KW: confirm this is okay with John - remove 36 individuals with only baseline data*

*KW: Remove partners too?*

In [ ]:
dyad_ids <- baseline_values |> 
  mutate(dyad_id = str_sub(study_id, 1, 3)) |> 
  filter(!study_id %in% subset(data, time != 0)$study_id) |> 
  pull(dyad_id) |> 
  unique()
  
data |> 
  mutate(dyad_id = str_sub(study_id, 1, 3)) |> 
  filter(dyad_id %in% dyad_ids) |> 
  select(study_id, dyad_id) |> 
  unique() |> 
  arrange(dyad_id) |> 
  print(n = Inf)


# A tibble: 53 × 2
   study_id dyad_id
   <chr>    <chr>  
 1 110FAM   110    
 2 110PT    110    
 3 111FAM   111    
 4 111PT    111    
 5 117PT    117    
 6 117FAM   117    
 7 119FAM   119    
 8 119PT    119    
 9 121FAM   121    
10 121PT    121    
11 125FAM   125    
12 125PT    125    
13 130FAM   130    
14 130PT    130    
15 139FAM   139    
16 139PT    139    
17 140PT    140    
18 140FAM   140    
19 148FAM   148    
20 148PT    148    
21 149FAM   149    
22 149PT    149    
23 155FAM   155    
24 155PT    155    
25 156PT    156    
26 156FAM   156    
27 167FAM   167    
28 181FAM   181    
29 181PT    181    
30 190FAM   190    
31 190PT    190    
32 198FAM   198    
33 198PT    198    
34 199FAM   199    
35 199PT    199    
36 203FAM   203    
37 203PT    203    
38 205FAM   205    
39 205PT    205    
40 209FAM   209    
41 209PT    209    
42 303FAM   303    
43 303PT    303    
44 314FAM   314    
45 314PT    314    
46 315PT    315    
47 315FAM   315    
4

Center baseline values of outcomes

In [ ]:
data <- data |> 
  mutate(hdd_0 = hdd_0 - mean(hdd_0, na.rm = TRUE),
         pda_0 = pda_0 - mean(pda_0, na.rm = TRUE),
         oq45_0 = oq45_0 - mean(oq45_0, na.rm = TRUE),
         relationsatisf_0 = relationsatisf_0 - mean(relationsatisf_0, na.rm = TRUE),
         abuse_0 = abuse_0 - mean(abuse_0, na.rm = TRUE))


Center time at the end of the intervention period (8 months).

In [ ]:
sjlabelled::get_labels(data$time)


[1] "Baseline" "4 Month"  "8 Month"  "12 Month"

 data$time   n   percent
         1 328 0.3534483
         2 315 0.3394397
         3 285 0.3071121

Create integer variable for heavy drinking days and days abstinent (baseline and followup values) - needed for negative binomial model

In [ ]:
data <- data |> 
  mutate(r_hdd = round(per_heavy_drink_days),
         r_hdd_0 = round(hdd_0),
         r_pda = round(per_days_abstinent),
         r_pda_0 = round(pda_0))


### EDA

Remove vars not used in analyses (e.g., keep scale score and remove individual items).

*KW: phq, promis, loneliness, coping, interaction scales not scored or in measures doc. Removed from analyses data set.*

In [ ]:
data <- data |> 
  select(-c(treatmentskills_1:treatmentskills_18_recode,
            oq45_1:oq45_45, alcoholproblems_1:alcoholproblems_15, 
            relationsatisf_1:relationsatisf_8, phq_1:phq_8, 
            abuse_1:abuse_16_yes, promis29_1:promis29_8, 
            abuse_physical_scored:abuse_psychological_scored, gender_open,
            loneliness_1:loneliness_8, healthservice_1, healthservice_2,
            healthservice_3,healthservice_4,healthservice_5, 
            healthservice_5a_1:healthservice_5c, 
            healthservice_6a_1:healthservice_8a, healthservice_8b, 
            healthservice_9a, healthservice_9b, bonding_1:bonding_5, 
            socrates_1:socrates_20, coping_1:coping_25, 
            drinkinggoals:drinkinggoal_7_text, covid_1:covid_3f,
            interaction_1:interaction_20, drink_druguse_1:drinkdruguse_8d,
            abuse_0, employmentstatus:notemployed_open, ethnicity:race_open, 
            livewith_1:livewith_open, meetings_yn, er_yn, outpatient_yn, inpatient_yn,
            n_days, mean_drinks_per_day, per_drink_days, covid_scored,
            redcap_event_name, recruitment_id)) |> 
  rename(other_medications_yn = healthservice_6,
         readmits_yn = r_eadmits_yn) |>  # renaming for descriptive/consistency
  relocate(study_id) |> 
  glimpse()


Rows: 928
Columns: 45
$ study_id                    <chr> "100FAM", "100FAM", "100FAM", "100PT", "10…
$ dyad                        <dbl+lbl> 1, 1, 1, 2, 2, 2, 1, 1, 1, 2, 2, 2, 1,…
$ arm                         <dbl+lbl> 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 2,…
$ group                       <dbl+lbl> 4, 4, 4, 1, 1, 1, 3, 3, 3, 0, 0, 0, 5,…
$ time_string                 <chr> "4 Month", "8 Month", "12 Month", "12 Mont…
$ time                        <dbl+lbl> 1, 2, 3, 3, 1, 2, 1, 2, 3, 1, 2, 3, 1,…
$ comments                    <chr> "", "", "", "", "", "", "", "", "", "", ""…
$ gender                      <dbl+lbl> 1, 1, 1, 2, 2, 2, 1, 1, 1, 2, 2, 2, 1,…
$ age                         <dbl> 55, 55, 55, 56, 56, 56, 62, 62, 62, 66, 66…
$ edu                         <dbl+lbl> 4, 4, 4, 3, 3, 3, 4, 4, 4, 3, 3, 3, 6,…
$ income                      <dbl+lbl> 3, 3, 3, 3, 3, 3, 4, 4, 4, 4, 4, 4, 4,…
$ partner_relationship        <dbl+lbl> 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 5,…
$ employment      

Number of participants

In [ ]:
data |> 
  pull(study_id) |>
  unique() |> 
  length() # 340


[1] 340

[1] 170

[1] 170

Complete dyad (patient + partner) for all participants

In [ ]:
data |> 
  select(study_id) |> 
  unique() |> 
  mutate(dyad_id = str_sub(study_id, 1, 3)) |> 
  group_by(dyad_id) |> 
  count() |> 
  filter(n != 2)


# A tibble: 0 × 2
# Groups:   dyad_id [0]
# ℹ 2 variables: dyad_id <chr>, n <int>

Missing data

In [ ]:
data |> 
  naniar::miss_var_summary() |> 
  print(n = Inf)


# A tibble: 45 × 3
   variable                    n_miss pct_miss
   <chr>                        <int>    <num>
 1 readmits_yn                    903   97.3  
 2 abuse_scored                   680   73.3  
 3 per_days_abstinent             498   53.7  
 4 per_heavy_drink_days           498   53.7  
 5 r_hdd                          498   53.7  
 6 r_pda                          498   53.7  
 7 socrates_scored                493   53.1  
 8 socrates_recognition_scored    493   53.1  
 9 socrates_ambivalence_scored    493   53.1  
10 treatmentskills_scored         479   51.6  
11 hdd_0                          464   50    
12 pda_0                          464   50    
13 r_hdd_0                        464   50    
14 r_pda_0                        464   50    
15 days_app_use                   455   49.0  
16 other_medications_yn            42    4.53 
17 mat_yn                          39    4.20 
18 peersupport_scored              39    4.20 
19 other_psych_treatment_yn        36    

Missing data for just patients

-   treatmentskills not applicable to patients
-   abuse and readmits have high missing data
-   Why days_app_use missing so high?

In [ ]:
data |> 
  filter(dyad == 2) |> 
  naniar::miss_var_summary() |> 
  print(n = Inf)


# A tibble: 45 × 3
   variable                    n_miss pct_miss
   <chr>                        <int>    <num>
 1 treatmentskills_scored         464  100    
 2 readmits_yn                    440   94.8  
 3 abuse_scored                   370   79.7  
 4 days_app_use                   151   32.5  
 5 per_days_abstinent              34    7.33 
 6 per_heavy_drink_days            34    7.33 
 7 r_hdd                           34    7.33 
 8 r_pda                           34    7.33 
 9 other_medications_yn            30    6.47 
10 mat_yn                          29    6.25 
11 socrates_scored                 29    6.25 
12 socrates_recognition_scored     29    6.25 
13 socrates_ambivalence_scored     29    6.25 
14 peersupport_scored              28    6.03 
15 other_psych_treatment_yn        26    5.60 
16 relationship_status             25    5.39 
17 oq45_scored                     25    5.39 
18 relationsatisf_scored           25    5.39 
19 income                          12    

Remove `readmits_yn` as covariate due to low endorsement

*KW: not sure why so many NAs*

In [ ]:
sjlabelled::get_labels(data$readmits_yn)


[1] "No"  "Yes"

Missing data for partners

In [ ]:
data |> 
  filter(dyad == 1) |> 
  naniar::miss_var_summary() |> 
  print(n = Inf)


# A tibble: 44 × 3
   variable                    n_miss pct_miss
   <chr>                        <int>    <num>
 1 per_days_abstinent             464  100    
 2 per_heavy_drink_days           464  100    
 3 socrates_scored                464  100    
 4 socrates_recognition_scored    464  100    
 5 socrates_ambivalence_scored    464  100    
 6 hdd_0                          464  100    
 7 pda_0                          464  100    
 8 r_hdd                          464  100    
 9 r_hdd_0                        464  100    
10 r_pda                          464  100    
11 r_pda_0                        464  100    
12 abuse_scored                   310   66.8  
13 days_app_use                   304   65.5  
14 treatmentskills_scored          15    3.23 
15 other_medications_yn            12    2.59 
16 peersupport_scored              11    2.37 
17 other_psych_treatment_yn        10    2.16 
18 mat_yn                          10    2.16 
19 relationship_status             10    

Participants in each arm

In [ ]:
data |> 
  group_by(study_id) |> 
  slice(1) |> 
  tabyl(arm)


 arm   n   percent
   0 108 0.3176471
   1 118 0.3470588
   2 114 0.3352941

Check stratification variables in arms - patient gender identity and alcohol use severity (moderate or severe AUD)

DSM5 criteria: 1 = severe, 2 = moderate, 3 = mild  
Group: 0 = patient control, 1 = patient achess, 2 = patient famchess,

In [ ]:
data |> 
  filter(dyad == 2) |> 
  group_by(study_id) |> 
  slice(1) |> 
  tabyl(group, gender)


 group  1  2 NA_
     0 22 31   1
     1 26 33   0
     2 23 34   0

 group  1 2 3 NA_
     0 51 1 1   1
     1 58 1 0   0
     2 50 6 1   0

Demographic and study related variables

Time string matches time

In [ ]:
data |> 
  tabyl(time, time_string)


 time 12 Month 4 Month 8 Month
    1        0     328       0
    2        0       0     315
    3      285       0       0

*KW: discuss with John if we need to do anything with these notes*

In [ ]:
data |> 
  filter(!comments == "") |> 
  pull(comments)


 [1] "Missing TLFB"                                                                                                                                                                                       
 [2] "Missing TLFB"                                                                                                                                                                                       
 [3] "Missing survey data"                                                                                                                                                                                
 [4] "Missing TLFB"                                                                                                                                                                                       
 [5] "Missing TLFB"                                                                                                                                                                         

Income

In [ ]:
sjlabelled::get_labels(data$income)


[1] "Less than $25,000"    "$25,000 to $34,999"   "$35,000 to $49,999"  
[4] "$50,000 to $74,999"   "$75,000 to $99,999"   "$100,000 to $149,999"
[7] "$150,000 to $199,999" "$200,000 or more"    

 data$income   n    percent valid_percent
           1 177 0.19073276    0.19386637
           2  57 0.06142241    0.06243154
           3  91 0.09806034    0.09967141
           4 192 0.20689655    0.21029573
           5 160 0.17241379    0.17524644
           6 144 0.15517241    0.15772180
           7  65 0.07004310    0.07119387
           8  27 0.02909483    0.02957284
          NA  15 0.01616379            NA

Partner relationship

In [ ]:
sjlabelled::get_labels(data$partner_relationship)


[1] "Romantic partner/spouse" "Parent"                 
[3] "Adult child"             "Other family member"    
[5] "Friend"                  "Recovery Coach"         

 data$partner_relationship   n     percent valid_percent
                         1 614 0.661637931   0.663783784
                         2  74 0.079741379   0.080000000
                         3  73 0.078663793   0.078918919
                         4  98 0.105603448   0.105945946
                         5  63 0.067887931   0.068108108
                         6   3 0.003232759   0.003243243
                        NA   3 0.003232759            NA

Employment

In [ ]:
sjlabelled::get_labels(data$employment)


[1] "No"  "Yes"

 data$employment   n     percent valid_percent
               0 315 0.339439655     0.3405405
               1 610 0.657327586     0.6594595
              NA   3 0.003232759            NA

Relationship status with partner

In [ ]:
sjlabelled::get_labels(data$relationship_status)


[1] "Still together (e.g., as a romantic couple, as family, as friends)"
[2] "Split up but still working on this project together"               
[3] "Split up and not working on this project together"                 
[4] "Other"                                                             

 data$relationship_status   n    percent valid_percent
                        1 762 0.82112069    0.85330347
                        2  62 0.06681034    0.06942889
                        3  24 0.02586207    0.02687570
                        4  45 0.04849138    0.05039194
                       NA  35 0.03771552            NA

 [1] "Still friends but he is unsure how much if at all she is using the app"                                                                         
 [2] "parent"                                                                                                                                         
 [3] "None"                                                                                                                                           
 [4] "None"                                                                                                                                           
 [5] "Not seeing"                                                                                                                                     
 [6] "Little contact"                                                                                                                                 
 [7] "work schedules collide"                                                                 

univariate descriptives of covariates, mediators, moderators and outcome variables

In [ ]:
data |> 
  select(age, edu, days_app_use:r_pda_0, -gender_original, -age_c, 
         -relationship_status, - relationship_status_open, -c(time_c:r_pda_0)) |> 
  skimr::skim()


  ----------------------------------------------------------------------------------------------------------------------------
  skim_variable                   n_missing   complete_rate    mean      sd       p0      p25     p50     p75     p100 hist
  ----------------------------- ----------- --------------- ------- ------- -------- -------- ------- ------- -------- -------
  age                                     6            0.99   47.62   13.59    21.00    37.00   47.50   58.00    89.00 ▅▇▇▃▁

  edu                                     3            1.00    4.28    1.08     1.00     4.00    4.00    5.00     6.00 ▁▅▇▅▃

  days_app_use                          455            0.51   41.14   38.05     0.00     6.00   32.00   74.00   161.00 ▇▃▃▂▁

  per_days_abstinent                    498            0.46   60.96   37.45     0.00    29.98   71.05   98.20   100.00 ▅▂▂▃▇

  per_heavy_drink_days                  498            0.46   24.20   33.27     0.00     0.00    5.65   34.35   100.00 ▇▂▁▁▂

  outpatient_yn_ever                      0            1.00    0.18    0.39     0.00     0.00    0.00    0.00     1.00 ▇▁▁▁▂

  other_psych_treatment_yn               36            0.96    0.16    0.37     0.00     0.00    0.00    0.00     1.00 ▇▁▁▁▂

  er_yn_ever                              0            1.00    0.22    0.41     0.00     0.00    0.00    0.00     1.00 ▇▁▁▁▂

  mat_yn                                 39            0.96    0.10    0.30     0.00     0.00    0.00    0.00     1.00 ▇▁▁▁▁

  other_medications_yn                   42            0.95    0.35    0.48     0.00     0.00    0.00    1.00     1.00 ▇▁▁▁▅

  inpatient_yn_ever                       0            1.00    0.10    0.30     0.00     0.00    0.00    0.00     1.00 ▇▁▁▁▁

  oq45_scored                            35            0.96   55.33   25.33     0.00    37.00   54.00   71.00   140.00 ▃▇▆▂▁

  relationsatisf_scored                  35            0.96   21.64    6.40     0.00    18.00   22.00   26.00    36.00 ▁▂▆▇▃

  abuse_scored                          680            0.27    4.40    7.35     0.00     1.00    2.00    6.00    72.00 ▇▁▁▁▁

  treatmentskills_scored                479            0.48    3.11    0.51     1.00     2.83    3.13    3.47     4.00 ▁▁▃▇▅

  peersupport_scored                     39            0.96    2.92    1.17     1.00     2.00    2.80    3.80     5.00 ▆▇▇▅▅

  socrates_scored                       493            0.47   32.36    7.06    10.00    28.00   33.00   39.00    40.00 ▁▁▃▅▇

  socrates_recognition_scored           493            0.47   25.84    6.70     7.00    22.00   26.00   31.00    35.00 ▁▂▆▇▇

  socrates_ambivalence_scored           493            0.47   13.48    4.25     4.00    11.00   14.00   17.00    20.00 ▂▃▅▇▆

  race_white_only                         3            1.00    0.76    0.43     0.00     1.00    1.00    1.00     1.00 ▂▁▁▁▇

  hdd_0                                 464            0.50    0.00   36.11   -49.00   -32.70   -6.90   39.70    51.00 ▇▆▃▃▇

  pda_0                                 464            0.50    0.00   33.84   -37.48   -35.81   -4.18   30.32    62.52 ▇▂▂▃▂

  oq45_0                                  3            1.00    0.00   24.41   -55.71   -18.71   -1.71   15.29    73.29 ▂▇▇▃▁

  relationsatisf_0                        3            1.00    0.00    5.01   -18.52    -2.52    0.48    3.48    12.48 ▁▂▇▇▂
  ----------------------------------------------------------------------------------------------------------------------------


Bivariate correlations for Patient

*KW: simplify?*

Notes:

patient primary outcome = `per_heavy_drink_days`  
partner primary outcome = `oq45_scored`

patient secondary outcome = `per_days_abstinent`  
partner secondary outcomes = `relationsatisf_scored` and `abuse_scored`

In [ ]:
data |> 
  filter(dyad == 2) |> 
  select(age, edu, days_app_use:r_pda_0, -gender_original, -age_c,
         -treatmentskills_scored, -relationship_status, 
         -relationship_status_open, -c(time_c:r_pda_0)) |> 
  cor(use = "pairwise.complete.obs") |> 
  round(2)


                              age   edu days_app_use per_days_abstinent
age                          1.00  0.04         0.21              -0.16
edu                          0.04  1.00        -0.01               0.00
days_app_use                 0.21 -0.01         1.00               0.02
per_days_abstinent          -0.16  0.00         0.02               1.00
per_heavy_drink_days         0.03 -0.13        -0.14              -0.75
outpatient_yn_ever          -0.25 -0.05        -0.19               0.18
other_psych_treatment_yn    -0.04  0.03         0.04               0.09
er_yn_ever                  -0.01 -0.22        -0.01               0.06
mat_yn                      -0.10 -0.02        -0.05               0.02
other_medications_yn         0.08  0.21        -0.13               0.00
inpatient_yn_ever           -0.18 -0.03        -0.19               0.15
oq45_scored                 -0.13 -0.15        -0.13              -0.28
relationsatisf_scored       -0.12  0.09         0.09            

Look for significant covariates (We will only include covariates that significantly predict the outcome).

List of covariate variable names: race_white_only, age_c, edu_c, meetings_yn, outpatient_yn_ever, er_yn_ever, mat_yn, inpatient_yn_ever, other_psych_treatment_yn, other_medications_yn

*KW: COVID acute/residual symptoms was originally listed as covariate but is removed per measures doc (Participants interpreted question differently).*

Check distribution of residuals

### Save out processed data for analyses

In [ ]:
data |> 
  write_csv(here::here(path_data, "famchess_data_ana.csv"))
